In [12]:
# DID NOT REALLY WORK!

import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
from typing import Tuple, Dict
from dataset import WallSample, create_wall_dataloader
from tqdm.auto import tqdm
import time
from datetime import datetime
from pathlib import Path
from lightly.utils.scheduler import cosine_schedule

from evaluator import ProbingEvaluator

def load_data(device):
    data_path = "/drive_reader/as16386/DL24FA"

    probe_train_ds = create_wall_dataloader(
        data_path=f"{data_path}/probe_normal/train",
        probing=True,
        device=device,
        train=True,
    )

    probe_val_normal_ds = create_wall_dataloader(
        data_path=f"{data_path}/probe_normal/val",
        probing=True,
        device=device,
        train=False,
    )

    probe_val_wall_ds = create_wall_dataloader(
        data_path=f"{data_path}/probe_wall/val",
        probing=True,
        device=device,
        train=False,
    )

    probe_val_ds = {"normal": probe_val_normal_ds, "wall": probe_val_wall_ds}

    return probe_train_ds, probe_val_ds

def evaluate_model(device, model, probe_train_ds, probe_val_ds):
    evaluator = ProbingEvaluator(
        device=device,
        model=model,
        probe_train_ds=probe_train_ds,
        probe_val_ds=probe_val_ds,
        quick_debug=False,
    )

    prober = evaluator.train_pred_prober()

    avg_losses = evaluator.evaluate_all(prober=prober)

    for probe_attr, loss in avg_losses.items():
        print(f"{probe_attr} loss: {loss}")

probe_train_ds, probe_val_ds = load_data("cuda")

class Encoder(nn.Module):
    def __init__(self, input_channels=2):
        super().__init__()
        # First conv: 65x65 -> 22x22
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=5, stride=3, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        
        # Second conv: 22x22 -> 8x8
        self.conv2 = nn.Conv2d(8, 32, kernel_size=3, stride=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.repr_dim = 32 * 8 * 8  # Full flattened representation
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # -> 22x22
        x = F.relu(self.bn2(self.conv2(x)))  # -> 8x8
        return x

class TransitionModel(nn.Module):
    def __init__(self, hidden_dim=32):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Action embedding
        self.action_embed = nn.Sequential(
            nn.Conv2d(2, hidden_dim // 2, 1),
            nn.BatchNorm2d(hidden_dim // 2),
            nn.ReLU(),
            nn.Conv2d(hidden_dim // 2, hidden_dim, 1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU()
        )
        
        # Transition model
        self.transition = nn.Sequential(
            nn.Conv2d(hidden_dim * 2, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim)
        )
        
    def forward(self, state, action):
        B, _, H, W = state.shape
        action = action.view(B, 2, 1, 1).expand(-1, -1, H, W)
        action_embedding = self.action_embed(action)
        combined = torch.cat([state, action_embedding], dim=1)
        delta = self.transition(combined)
        next_state = state + delta
        return next_state

class ProjectionHead(nn.Module):
    def __init__(self, input_dim=32*8*8, hidden_dim=2048, output_dim=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        if len(x.shape) > 2:
            x = x.reshape(x.shape[0], -1)
        return self.net(x)

class PredictionHead(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, output_dim=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

class BYOLWorldModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Online networks
        self.encoder = Encoder(input_channels=2)
        self.predictor = TransitionModel(hidden_dim=32)
        self.projection = ProjectionHead(input_dim=32*8*8)
        self.prediction = PredictionHead()
        
        # Target networks
        self.encoder_momentum = copy.deepcopy(self.encoder)
        self.projection_momentum = copy.deepcopy(self.projection)
        
        # Deactivate gradients for momentum networks
        for param in self.encoder_momentum.parameters():
            param.requires_grad = False
        for param in self.projection_momentum.parameters():
            param.requires_grad = False
            
        self.repr_dim = self.encoder.repr_dim
        
    def forward_prediction(self, states, actions):
        B, _, _, H, W = states.shape
        T = actions.shape[1] + 1
        
        curr_state = self.encoder(states.squeeze(1))
        predictions = [curr_state]
        
        for t in range(T-1):
            curr_state = self.predictor(curr_state, actions[:, t])
            predictions.append(curr_state)
            
        predictions = torch.stack(predictions, dim=1)
        return predictions

    def forward(self, states, actions):
        init_states = states[:, 0:1]
        predictions = self.forward_prediction(init_states, actions)
        B, T, C, H, W = predictions.shape
        predictions = predictions.view(B, T, -1)
        return predictions

    def compute_byol_loss(self, pred_flat, target_flat):
        pred_proj = self.projection(pred_flat)
        pred_out = self.prediction(pred_proj)
        
        with torch.no_grad():
            target_proj = self.projection_momentum(target_flat)
        
        pred_out = F.normalize(pred_out, dim=1)
        target_proj = F.normalize(target_proj, dim=1)
        
        loss = 2 - 2 * (pred_out * target_proj).sum(dim=1).mean()
        
        return loss, {
            'byol_loss': loss.item(),
            'total_loss': loss.item()
        }

    def training_step(self, batch):
        states = batch.states
        actions = batch.actions
        
        # Get initial state
        init_states = states[:, 0:1]
        
        # Get predictions for all steps
        predictions = self.forward_prediction(init_states, actions)
        
        # Initialize losses
        total_loss = 0.0
        accumulated_losses = {
            'total_loss': 0.0,
            'byol_loss': 0.0
        }
        
        # Compute loss for each timestep
        for t in range(actions.shape[1]):
            pred_state = predictions[:, t+1]
            target_obs = states[:, t+1]
            
            with torch.no_grad():
                target_state = self.encoder_momentum(target_obs)
            
            pred_flat = pred_state.flatten(start_dim=1)
            target_flat = target_state.flatten(start_dim=1)
            
            loss, component_losses = self.compute_byol_loss(pred_flat, target_flat)
            
            total_loss += loss
            for k in accumulated_losses:
                accumulated_losses[k] += component_losses[k]
        
        # Average losses over timesteps
        total_loss = total_loss / actions.shape[1]
        for k in accumulated_losses:
            accumulated_losses[k] /= actions.shape[1]
        
        return total_loss, predictions, accumulated_losses
    
    @torch.no_grad()
    def update_target(self, current_epoch, total_epochs):
        tau = cosine_schedule(current_epoch, total_epochs, 0.996, 1.0)
        
        for online, target in zip(self.encoder.parameters(), self.encoder_momentum.parameters()):
            target.data = tau * target.data + (1 - tau) * online.data
            
        for online, target in zip(self.projection.parameters(), self.projection_momentum.parameters()):
            target.data = tau * target.data + (1 - tau) * online.data

def train_epoch(model, dataloader, optimizer, epoch, total_epochs):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    train_losses = {
        'total_loss': 0.0,
        'byol_loss': 0.0
    }
    
    # Progress bar for batches
    num_batches = len(dataloader)
    progress_bar = tqdm(enumerate(dataloader), 
                       total=num_batches,
                       desc=f'Epoch {epoch}/{total_epochs}',
                       leave=True)
    
    start_time = time.time()
    batch_times = []
    
    for batch_idx, batch in progress_bar:
        batch_start = time.time()
        
        # Move batch to GPU
        batch = batch._replace(
            states=batch.states.cuda(),
            actions=batch.actions.cuda(),
            locations=batch.locations.cuda() if batch.locations is not None else None
        )
        
        # Forward pass and compute loss
        optimizer.zero_grad()
        loss, _, component_losses = model.training_step(batch)  # Removed extra arguments
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Update momentum networks
        model.update_target(epoch, total_epochs)
        
        # Update timing
        batch_time = time.time() - batch_start
        batch_times.append(batch_time)
        
        # Update losses
        total_loss += loss.item()
        for k in train_losses:
            train_losses[k] += component_losses[k]
        
        # Update progress bar
        current_loss = total_loss / (batch_idx + 1)
        current_batch_time = sum(batch_times) / len(batch_times)
        
        progress_bar.set_postfix({
            'loss': f'{current_loss:.4f}',
            'time/batch': f'{current_batch_time:.3f}s',
            'gpu_mem': f'{torch.cuda.max_memory_allocated()/1e9:.1f}GB'
        })
    
    # Compute final metrics
    num_batches = len(dataloader)
    total_loss /= num_batches
    for k in train_losses:
        train_losses[k] /= num_batches
    
    epoch_time = time.time() - start_time
    avg_batch_time = sum(batch_times) / len(batch_times)
    
    return total_loss, train_losses, {
        'epoch_time': epoch_time,
        'avg_batch_time': avg_batch_time
    }

def train_model(model, train_loader, num_epochs=100, learning_rate=3e-4, 
                save_dir='checkpoints/byol', save_frequency=10):
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create checkpoint directory
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Training for {num_epochs} epochs")
    print(f"Checkpoints will be saved to {save_dir}")
    
    for epoch in range(num_epochs):
        # Train for one epoch
        train_loss, train_losses, timing_stats = train_epoch(
            model, train_loader, optimizer, epoch, num_epochs
        )
        
        # Print epoch summary
        print(f"\nEpoch {epoch}/{num_epochs} Summary:")
        print(f"Train Loss: {train_loss:.4f}")
        print("Component Losses:")
        for k, v in train_losses.items():
            print(f"  {k}: {v:.4f}")
        print(f"Epoch Time: {timing_stats['epoch_time']:.1f}s")
        print(f"Avg Batch Time: {timing_stats['avg_batch_time']:.3f}s")
        print(f"GPU Memory: {torch.cuda.max_memory_allocated() / 1024**2:.0f}MB")
        evaluate_model("cuda", model, probe_train_ds, probe_val_ds)
        
        # Save checkpoint
        if (epoch + 1) % save_frequency == 0:
            checkpoint_path = save_dir / f"checkpoint_epoch_{epoch+1}.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, checkpoint_path)
            print(f"Checkpoint saved to {checkpoint_path}")
        
        print("-" * 80)

if __name__ == "__main__":
    # Create model and data loader
    model = BYOLWorldModel().cuda()
    train_loader = create_wall_dataloader(
        "/drive_reader/as16386/DL24FA/train",  # Update with your path
        batch_size=128,
        train=True
    )
    
    # Start training
    train_model(
        model=model,
        train_loader=train_loader,
        num_epochs=100,
        learning_rate=3e-5,
        save_dir='byol_try_checkpoints/',
        save_frequency=10
    )

Starting training at 2024-12-11 23:38:42
Training for 100 epochs
Checkpoints will be saved to byol_try_checkpoints


Epoch 0/100: 100%|██████████| 1148/1148 [01:02<00:00, 18.43it/s, loss=0.3036, time/batch=0.029s, gpu_mem=1.8GB]



Epoch 0/100 Summary:
Train Loss: 0.3036
Component Losses:
  total_loss: 0.3036
  byol_loss: 0.3036
Epoch Time: 62.3s
Avg Batch Time: 0.029s
GPU Memory: 1683MB


Probe prediction epochs:   0%|          | 0/20 [00:00<?, ?it/s]

normalized pred locations loss 2.0951576232910156


...

normalized pred locations loss 0.7294135093688965


Eval probe pred: 100%|██████████| 62/62 [00:00<00:00, 82.51it/s]


normal loss: 249.85256958007812
wall loss: 188.7933349609375
--------------------------------------------------------------------------------


Epoch 3/100:  53%|█████▎    | 612/1148 [00:30<00:26, 19.96it/s, loss=0.0487, time/batch=0.029s, gpu_mem=1.8GB]


KeyboardInterrupt: 

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
from typing import Tuple, Dict, NamedTuple
from tqdm.auto import tqdm
import time
from datetime import datetime
from pathlib import Path

class Encoder(nn.Module):
    def __init__(self, input_channels=2):
        super().__init__()
        # First conv: 65x65 -> 22x22
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=5, stride=3, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        
        # Second conv: 22x22 -> 8x8
        self.conv2 = nn.Conv2d(8, 32, kernel_size=3, stride=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.repr_dim = 32 * 8 * 8
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # -> 22x22
        x = F.relu(self.bn2(self.conv2(x)))  # -> 8x8
        return x

class TransitionModel(nn.Module):
    def __init__(self, hidden_dim=32):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Action embedding
        self.action_embed = nn.Sequential(
            nn.Conv2d(2, hidden_dim // 2, 1),
            nn.BatchNorm2d(hidden_dim // 2),
            nn.ReLU(),
            nn.Conv2d(hidden_dim // 2, hidden_dim, 1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU()
        )
        
        # Transition model
        self.transition = nn.Sequential(
            nn.Conv2d(hidden_dim * 2, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim)
        )
        
    def forward(self, state, action):
        B, _, H, W = state.shape
        action = action.view(B, 2, 1, 1).expand(-1, -1, H, W)
        action_embedding = self.action_embed(action)
        combined = torch.cat([state, action_embedding], dim=1)
        delta = self.transition(combined)
        next_state = state + delta
        return next_state

class CombinedLoss(nn.Module):
    def __init__(self, 
                 vicreg_sim_coef=25.0,
                 vicreg_std_coef=25.0,
                 vicreg_cov_coef=1.0,
                 barlow_lambda=0.005,
                 loss_weight=0.5):  # Weight between VICReg and Barlow
        super().__init__()
        self.vicreg_sim_coef = vicreg_sim_coef
        self.vicreg_std_coef = vicreg_std_coef
        self.vicreg_cov_coef = vicreg_cov_coef
        self.barlow_lambda = barlow_lambda
        self.loss_weight = loss_weight
    
    def off_diagonal(self, x):
        n = x.shape[0]
        return x.flatten()[:-1].view(n-1, n+1)[:, 1:].flatten()
    
    def forward(self, z_a, z_b):
        N = z_a.shape[0]
        D = z_a.shape[1]
        
        # VICReg components
        sim_loss = F.mse_loss(z_a, z_b)
        
        std_z_a = torch.sqrt(z_a.var(dim=0) + 1e-04)
        std_z_b = torch.sqrt(z_b.var(dim=0) + 1e-04)
        std_loss = torch.mean(F.relu(1 - std_z_a)) + torch.mean(F.relu(1 - std_z_b))
        
        z_a_centered = z_a - z_a.mean(dim=0)
        z_b_centered = z_b - z_b.mean(dim=0)
        
        cov_z_a = (z_a_centered.T @ z_a_centered) / (N - 1)
        cov_z_b = (z_b_centered.T @ z_b_centered) / (N - 1)
        
        vicreg_cov_loss = (self.off_diagonal(cov_z_a).pow_(2).sum() / D +
                          self.off_diagonal(cov_z_b).pow_(2).sum() / D)
        
        # Barlow Twins components
        z_a_norm = (z_a - z_a.mean(dim=0)) / (z_a.std(dim=0) + 1e-6)
        z_b_norm = (z_b - z_b.mean(dim=0)) / (z_b.std(dim=0) + 1e-6)
        
        c = torch.mm(z_a_norm.T, z_b_norm) / N
        
        on_diag = torch.diagonal(c).add_(-1).pow_(2).sum()
        off_diag = torch.triu(c.pow(2), diagonal=1).sum() + torch.tril(c.pow(2), diagonal=-1).sum()
        barlow_loss = on_diag + self.barlow_lambda * off_diag
        
        # Combine VICReg losses
        vicreg_loss = (self.vicreg_sim_coef * sim_loss +
                      self.vicreg_std_coef * std_loss +
                      self.vicreg_cov_coef * vicreg_cov_loss)
        
        # Final weighted combination
        total_loss = self.loss_weight * vicreg_loss + (1 - self.loss_weight) * barlow_loss
        
        return total_loss, {
            'total_loss': total_loss.item(),
            'vicreg_sim_loss': sim_loss.item(),
            'vicreg_std_loss': std_loss.item(),
            'vicreg_cov_loss': vicreg_cov_loss.item(),
            'barlow_loss': barlow_loss.item()
        }

class WorldModelCombined(nn.Module):
    def __init__(self, vicreg_sim_coef=25.0, vicreg_std_coef=25.0, vicreg_cov_coef=1.0,
                 barlow_lambda=0.005, loss_weight=0.5):
        super().__init__()
        self.encoder = Encoder(input_channels=2)
        self.predictor = TransitionModel(hidden_dim=32)
        
        self.criterion = CombinedLoss(
            vicreg_sim_coef=vicreg_sim_coef,
            vicreg_std_coef=vicreg_std_coef,
            vicreg_cov_coef=vicreg_cov_coef,
            barlow_lambda=barlow_lambda,
            loss_weight=loss_weight
        )
        
        self.repr_dim = self.encoder.repr_dim
    
    def forward_prediction(self, states, actions):
        B, _, _, H, W = states.shape
        T = actions.shape[1] + 1
        
        curr_state = self.encoder(states.squeeze(1))
        predictions = [curr_state]
        
        for t in range(T-1):
            curr_state = self.predictor(curr_state, actions[:, t])
            predictions.append(curr_state)
            
        predictions = torch.stack(predictions, dim=1)
        return predictions

    def forward(self, states, actions):
        init_states = states[:, 0:1]
        predictions = self.forward_prediction(init_states, actions)
        B, T, C, H, W = predictions.shape
        predictions = predictions.view(B, T, -1)
        return predictions

    def training_step(self, batch):
        states = batch.states
        actions = batch.actions
        
        init_states = states[:, 0:1]
        predictions = self.forward_prediction(init_states, actions)
        
        total_loss = 0.0
        accumulated_losses = {
            'total_loss': 0.0,
            'vicreg_sim_loss': 0.0,
            'vicreg_std_loss': 0.0,
            'vicreg_cov_loss': 0.0,
            'barlow_loss': 0.0
        }
        
        for t in range(actions.shape[1]):
            pred_state = predictions[:, t+1]
            target_obs = states[:, t+1]
            
            target_state = self.encoder(target_obs)
            
            pred_flat = pred_state.flatten(start_dim=1)
            target_flat = target_state.flatten(start_dim=1)
            
            loss, component_losses = self.criterion(pred_flat, target_flat)
            
            total_loss += loss
            for k in accumulated_losses:
                accumulated_losses[k] += component_losses[k]
        
        total_loss = total_loss / actions.shape[1]
        for k in accumulated_losses:
            accumulated_losses[k] /= actions.shape[1]
        
        return total_loss, predictions, accumulated_losses

def train_epoch(model, dataloader, optimizer, epoch, total_epochs):
    model.train()
    total_loss = 0.0
    train_losses = {
        'total_loss': 0.0,
        'vicreg_sim_loss': 0.0,
        'vicreg_std_loss': 0.0,
        'vicreg_cov_loss': 0.0,
        'barlow_loss': 0.0
    }
    
    progress_bar = tqdm(enumerate(dataloader), 
                       total=len(dataloader),
                       desc=f'Epoch {epoch}/{total_epochs}',
                       leave=True)
    
    start_time = time.time()
    batch_times = []
    
    for batch_idx, batch in progress_bar:
        batch_start = time.time()
        
        batch = batch._replace(
            states=batch.states.cuda(),
            actions=batch.actions.cuda(),
            locations=batch.locations.cuda() if batch.locations is not None else None
        )
        
        optimizer.zero_grad()
        loss, _, component_losses = model.training_step(batch)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        batch_time = time.time() - batch_start
        batch_times.append(batch_time)
        
        total_loss += loss.item()
        for k in train_losses:
            train_losses[k] += component_losses[k]
        
        current_loss = total_loss / (batch_idx + 1)
        current_batch_time = sum(batch_times) / len(batch_times)
        
        progress_bar.set_postfix({
            'loss': f'{current_loss:.4f}',
            'time/batch': f'{current_batch_time:.3f}s',
            'gpu_mem': f'{torch.cuda.max_memory_allocated()/1e9:.1f}GB'
        })
    
    num_batches = len(dataloader)
    total_loss /= num_batches
    for k in train_losses:
        train_losses[k] /= num_batches
    
    epoch_time = time.time() - start_time
    avg_batch_time = sum(batch_times) / len(batch_times)
    
    return total_loss, train_losses, {
        'epoch_time': epoch_time,
        'avg_batch_time': avg_batch_time
    }

def train_model(model, train_loader, num_epochs=100, learning_rate=3e-4, 
                save_dir='checkpoints/combined', save_frequency=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Training for {num_epochs} epochs")
    print(f"Checkpoints will be saved to {save_dir}")
    
    for epoch in range(num_epochs):
        train_loss, train_losses, timing_stats = train_epoch(
            model, train_loader, optimizer, epoch, num_epochs
        )
        
        print(f"\nEpoch {epoch}/{num_epochs} Summary:")
        print(f"Train Loss: {train_loss:.4f}")
        print("Component Losses:")
        for k, v in train_losses.items():
            print(f"  {k}: {v:.4f}")
        print(f"Epoch Time: {timing_stats['epoch_time']:.1f}s")
        print(f"Avg Batch Time: {timing_stats['avg_batch_time']:.3f}s")
        print(f"GPU Memory: {torch.cuda.max_memory_allocated() / 1024**2:.0f}MB")
        
        if (epoch + 1) % save_frequency == 0:
            checkpoint_path = save_dir / f"checkpoint_epoch_{epoch+1}.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, checkpoint_path)
            print(f"Checkpoint saved to {checkpoint_path}")
        
        evaluate_model("cuda", model, probe_train_ds, probe_val_ds)
        
        print("-" * 80)

if __name__ == "__main__":
    from dataset import create_wall_dataloader, WallSample
    
    # Create model with combined losses
    model = WorldModelCombined(
        vicreg_sim_coef=25.0,
        vicreg_std_coef=25.0,
        vicreg_cov_coef=1.0,
        barlow_lambda=0.005,
        loss_weight=0.5  # Equal weight between VICReg and Barlow
    ).cuda()
    
    # Create data loader
    train_loader = create_wall_dataloader(
        "/drive_reader/as16386/DL24FA/train",  # Update with your path
        batch_size=128,
        train=True
    )
    
    # Start training
    train_model(
        model=model,
        train_loader=train_loader,
        num_epochs=100,
        learning_rate=3e-5,
        save_dir='checkpoints/combined_barlow_vicreg',
        save_frequency=10
    )

Starting training at 2024-12-11 23:46:02
Training for 100 epochs
Checkpoints will be saved to checkpoints/combined_barlow_vicreg


Epoch 0/100: 100%|██████████| 1148/1148 [01:38<00:00, 11.66it/s, loss=3203.5392, time/batch=0.060s, gpu_mem=2.1GB]



Epoch 0/100 Summary:
Train Loss: 3203.5392
Component Losses:
  total_loss: 3203.5393
  vicreg_sim_loss: 19.9145
  vicreg_std_loss: 1.1452
  vicreg_cov_loss: 4325.8279
  barlow_loss: 1554.7584
Epoch Time: 98.4s
Avg Batch Time: 0.060s
GPU Memory: 1988MB


Probe prediction epochs:   0%|          | 0/20 [00:00<?, ?it/s]

normalized pred locations loss 0.9376960396766663


...

normalized pred locations loss 0.007609501481056213


Eval probe pred: 100%|██████████| 62/62 [00:02<00:00, 21.79it/s]


normal loss: 2.9689748287200928
wall loss: 6.159099578857422
--------------------------------------------------------------------------------


Epoch 13/100:  65%|██████▍   | 744/1148 [02:07<01:09,  5.82it/s, loss=295.4510, time/batch=0.082s, gpu_mem=2.1GB]


KeyboardInterrupt: 

In [ ]:
# TRYING LOSS NORMALIZATION

import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
from typing import Tuple, Dict, NamedTuple
from tqdm.auto import tqdm
import time
from datetime import datetime
from pathlib import Path

class Encoder(nn.Module):
    def __init__(self, input_channels=2):
        super().__init__()
        # First conv: 65x65 -> 22x22
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=5, stride=3, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        
        # Second conv: 22x22 -> 8x8
        self.conv2 = nn.Conv2d(8, 32, kernel_size=3, stride=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.repr_dim = 32 * 8 * 8
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # -> 22x22
        x = F.relu(self.bn2(self.conv2(x)))  # -> 8x8
        return x

class TransitionModel(nn.Module):
    def __init__(self, hidden_dim=32):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Action embedding
        self.action_embed = nn.Sequential(
            nn.Conv2d(2, hidden_dim // 2, 1),
            nn.BatchNorm2d(hidden_dim // 2),
            nn.ReLU(),
            nn.Conv2d(hidden_dim // 2, hidden_dim, 1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU()
        )
        
        # Transition model
        self.transition = nn.Sequential(
            nn.Conv2d(hidden_dim * 2, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1),
            nn.BatchNorm2d(hidden_dim)
        )
        
    def forward(self, state, action):
        B, _, H, W = state.shape
        action = action.view(B, 2, 1, 1).expand(-1, -1, H, W)
        action_embedding = self.action_embed(action)
        combined = torch.cat([state, action_embedding], dim=1)
        delta = self.transition(combined)
        next_state = state + delta
        return next_state

class CombinedLoss(nn.Module):
    def __init__(self, 
                 vicreg_sim_coef=25.0,
                 vicreg_std_coef=25.0,
                 vicreg_cov_coef=1.0,
                 barlow_lambda=0.005,
                 loss_weight=0.5,
                 normalize_losses=True):
        super().__init__()
        self.vicreg_sim_coef = vicreg_sim_coef
        self.vicreg_std_coef = vicreg_std_coef
        self.vicreg_cov_coef = vicreg_cov_coef
        self.barlow_lambda = barlow_lambda
        self.loss_weight = loss_weight
        self.normalize_losses = normalize_losses
        
        # Running statistics for normalization
        self.register_buffer('vicreg_running_mean', torch.tensor(0.0))
        self.register_buffer('barlow_running_mean', torch.tensor(0.0))
        self.momentum = 0.9  # For updating running means
        
    def off_diagonal(self, x):
        n = x.shape[0]
        return x.flatten()[:-1].view(n-1, n+1)[:, 1:].flatten()
    
    def update_running_means(self, vicreg_loss, barlow_loss):
        if self.training:
            self.vicreg_running_mean = self.momentum * self.vicreg_running_mean + \
                                     (1 - self.momentum) * vicreg_loss.detach()
            self.barlow_running_mean = self.momentum * self.barlow_running_mean + \
                                     (1 - self.momentum) * barlow_loss.detach()
    
    def normalize_loss(self, vicreg_loss, barlow_loss):
        # Prevent division by zero
        eps = 1e-6
        
        # Use running means to normalize if they're non-zero
        if self.vicreg_running_mean > eps and self.barlow_running_mean > eps:
            vicreg_norm = vicreg_loss / self.vicreg_running_mean
            barlow_norm = barlow_loss / self.barlow_running_mean
        else:
            # Initially, scale based on typical ranges
            vicreg_norm = vicreg_loss / 17.5  # Assuming typical VICReg total around 17.5
            barlow_norm = barlow_loss / 5.0   # Assuming typical Barlow total around 5.0
            
        return vicreg_norm, barlow_norm
    
    def forward(self, z_a, z_b):
        N = z_a.shape[0]
        D = z_a.shape[1]
        
        # VICReg components
        sim_loss = F.mse_loss(z_a, z_b)
        
        std_z_a = torch.sqrt(z_a.var(dim=0) + 1e-04)
        std_z_b = torch.sqrt(z_b.var(dim=0) + 1e-04)
        std_loss = torch.mean(F.relu(1 - std_z_a)) + torch.mean(F.relu(1 - std_z_b))
        
        z_a_centered = z_a - z_a.mean(dim=0)
        z_b_centered = z_b - z_b.mean(dim=0)
        
        cov_z_a = (z_a_centered.T @ z_a_centered) / (N - 1)
        cov_z_b = (z_b_centered.T @ z_b_centered) / (N - 1)
        
        vicreg_cov_loss = (self.off_diagonal(cov_z_a).pow_(2).sum() / D +
                          self.off_diagonal(cov_z_b).pow_(2).sum() / D)
        
        # Combine VICReg components
        vicreg_loss = (self.vicreg_sim_coef * sim_loss +
                      self.vicreg_std_coef * std_loss +
                      self.vicreg_cov_coef * vicreg_cov_loss)
        
        # Barlow Twins components
        z_a_norm = (z_a - z_a.mean(dim=0)) / (z_a.std(dim=0) + 1e-6)
        z_b_norm = (z_b - z_b.mean(dim=0)) / (z_b.std(dim=0) + 1e-6)
        
        c = torch.mm(z_a_norm.T, z_b_norm) / N
        
        on_diag = torch.diagonal(c).add_(-1).pow_(2).sum()
        off_diag = torch.triu(c.pow(2), diagonal=1).sum() + torch.tril(c.pow(2), diagonal=-1).sum()
        barlow_loss = on_diag + self.barlow_lambda * off_diag
        
        # Update running means
        self.update_running_means(vicreg_loss, barlow_loss)
        
        # Normalize losses if enabled
        if self.normalize_losses:
            vicreg_loss, barlow_loss = self.normalize_loss(vicreg_loss, barlow_loss)
        
        # Combine normalized losses
        total_loss = self.loss_weight * vicreg_loss + (1 - self.loss_weight) * barlow_loss
        
        return total_loss, {
            'total_loss': total_loss.item(),
            'vicreg_sim_loss': sim_loss.item(),
            'vicreg_std_loss': std_loss.item(),
            'vicreg_cov_loss': vicreg_cov_loss.item(),
            'barlow_loss': barlow_loss.item(),
            'vicreg_mean': self.vicreg_running_mean.item(),
            'barlow_mean': self.barlow_running_mean.item()
        }

class WorldModelCombined(nn.Module):
    def __init__(self, vicreg_sim_coef=25.0, vicreg_std_coef=25.0, vicreg_cov_coef=1.0,
                 barlow_lambda=0.005, loss_weight=0.5):
        super().__init__()
        self.encoder = Encoder(input_channels=2)
        self.predictor = TransitionModel(hidden_dim=32)
        
        self.criterion = CombinedLoss(
            vicreg_sim_coef=vicreg_sim_coef,
            vicreg_std_coef=vicreg_std_coef,
            vicreg_cov_coef=vicreg_cov_coef,
            barlow_lambda=barlow_lambda,
            loss_weight=loss_weight
        )
        
        self.repr_dim = self.encoder.repr_dim
    
    def forward_prediction(self, states, actions):
        B, _, _, H, W = states.shape
        T = actions.shape[1] + 1
        
        curr_state = self.encoder(states.squeeze(1))
        predictions = [curr_state]
        
        for t in range(T-1):
            curr_state = self.predictor(curr_state, actions[:, t])
            predictions.append(curr_state)
            
        predictions = torch.stack(predictions, dim=1)
        return predictions

    def forward(self, states, actions):
        init_states = states[:, 0:1]
        predictions = self.forward_prediction(init_states, actions)
        B, T, C, H, W = predictions.shape
        predictions = predictions.view(B, T, -1)
        return predictions

    def training_step(self, batch):
        states = batch.states
        actions = batch.actions
        
        init_states = states[:, 0:1]
        predictions = self.forward_prediction(init_states, actions)
        
        total_loss = 0.0
        accumulated_losses = {
            'total_loss': 0.0,
            'vicreg_sim_loss': 0.0,
            'vicreg_std_loss': 0.0,
            'vicreg_cov_loss': 0.0,
            'barlow_loss': 0.0
        }
        
        for t in range(actions.shape[1]):
            pred_state = predictions[:, t+1]
            target_obs = states[:, t+1]
            
            target_state = self.encoder(target_obs)
            
            pred_flat = pred_state.flatten(start_dim=1)
            target_flat = target_state.flatten(start_dim=1)
            
            loss, component_losses = self.criterion(pred_flat, target_flat)
            
            total_loss += loss
            for k in accumulated_losses:
                accumulated_losses[k] += component_losses[k]
        
        total_loss = total_loss / actions.shape[1]
        for k in accumulated_losses:
            accumulated_losses[k] /= actions.shape[1]
        
        return total_loss, predictions, accumulated_losses

def train_epoch(model, dataloader, optimizer, epoch, total_epochs):
    model.train()
    total_loss = 0.0
    train_losses = {
        'total_loss': 0.0,
        'vicreg_sim_loss': 0.0,
        'vicreg_std_loss': 0.0,
        'vicreg_cov_loss': 0.0,
        'barlow_loss': 0.0
    }
    
    progress_bar = tqdm(enumerate(dataloader), 
                       total=len(dataloader),
                       desc=f'Epoch {epoch}/{total_epochs}',
                       leave=True)
    
    start_time = time.time()
    batch_times = []
    
    for batch_idx, batch in progress_bar:
        batch_start = time.time()
        
        batch = batch._replace(
            states=batch.states.cuda(),
            actions=batch.actions.cuda(),
            locations=batch.locations.cuda() if batch.locations is not None else None
        )
        
        optimizer.zero_grad()
        loss, _, component_losses = model.training_step(batch)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        batch_time = time.time() - batch_start
        batch_times.append(batch_time)
        
        total_loss += loss.item()
        for k in train_losses:
            train_losses[k] += component_losses[k]
        
        current_loss = total_loss / (batch_idx + 1)
        current_batch_time = sum(batch_times) / len(batch_times)
        
        progress_bar.set_postfix({
            'loss': f'{current_loss:.4f}',
            'time/batch': f'{current_batch_time:.3f}s',
            'gpu_mem': f'{torch.cuda.max_memory_allocated()/1e9:.1f}GB'
        })
    
    num_batches = len(dataloader)
    total_loss /= num_batches
    for k in train_losses:
        train_losses[k] /= num_batches
    
    epoch_time = time.time() - start_time
    avg_batch_time = sum(batch_times) / len(batch_times)
    
    return total_loss, train_losses, {
        'epoch_time': epoch_time,
        'avg_batch_time': avg_batch_time
    }

def train_model(model, train_loader, num_epochs=100, learning_rate=3e-4, 
                save_dir='checkpoints/combined', save_frequency=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Training for {num_epochs} epochs")
    print(f"Checkpoints will be saved to {save_dir}")
    
    for epoch in range(num_epochs):
        train_loss, train_losses, timing_stats = train_epoch(
            model, train_loader, optimizer, epoch, num_epochs
        )
        
        print(f"\nEpoch {epoch}/{num_epochs} Summary:")
        print(f"Train Loss: {train_loss:.4f}")
        print("Component Losses:")
        for k, v in train_losses.items():
            print(f"  {k}: {v:.4f}")
        print(f"Epoch Time: {timing_stats['epoch_time']:.1f}s")
        print(f"Avg Batch Time: {timing_stats['avg_batch_time']:.3f}s")
        print(f"GPU Memory: {torch.cuda.max_memory_allocated() / 1024**2:.0f}MB")
        
        if (epoch + 1) % save_frequency == 0:
            checkpoint_path = save_dir / f"checkpoint_epoch_{epoch+1}.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, checkpoint_path)
            print(f"Checkpoint saved to {checkpoint_path}")
        
        evaluate_model("cuda", model, probe_train_ds, probe_val_ds)
        
        print("-" * 80)

if __name__ == "__main__":
    from dataset import create_wall_dataloader, WallSample
    
    # Create model with combined losses
    model = WorldModelCombined(
        vicreg_sim_coef=25.0,
        vicreg_std_coef=25.0,
        vicreg_cov_coef=1.0,
        barlow_lambda=0.005,
        loss_weight=0.5  # Equal weight between VICReg and Barlow
    ).cuda()
    
    # Create data loader
    train_loader = create_wall_dataloader(
        "/drive_reader/as16386/DL24FA/train",  # Update with your path
        batch_size=128,
        train=True
    )
    
    # Start training
    train_model(
        model=model,
        train_loader=train_loader,
        num_epochs=100,
        learning_rate=3e-5,
        save_dir='checkpoints/combined_barlow_vicreg_normalized',
        save_frequency=10
    )